In [ ]:
from google.colab import drive
drive.mount('/content/drive')
data_dir = "/content/drive/My Drive/Aerial classes small"  # Adjust the path


preprocessing

In [ ]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the dataset directory paths
train_dir = os.path.join(data_dir, "train")
val_dir = os.path.join(data_dir, "val")
test_dir = os.path.join(data_dir, "test")

# Create ImageDataGenerators
train_datagen = ImageDataGenerator(rescale=1.0/255.0,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   horizontal_flip=True)

val_test_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Load datasets from directories
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),  # Resize images to 128x128
    batch_size=32,          # Adjust batch size as needed
    class_mode='categorical'  # Multiclass classification
)

val_data = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical'
)

test_data = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical'
)

# Display class indices to ensure proper mapping
print("Class indices:", train_data.class_indices)


model 2, dropout 0.3

In [ ]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Create a CNN model
model2 = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(5, activation='softmax')  # 5 classes
])

# Compile the model
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model2.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)                    │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 28, 28, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │       3,211,392 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 5)                   │             645 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,305,285 (12.61 MB)

 Trainable params: 3,305,285 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Train the model
history = model2.fit(
    train_data,
    validation_data=val_data,
    epochs=50,  # You can adjust the number of epochs
    verbose=1
)


Epoch 1/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 15s 729ms/step - accuracy: 0.2047 - loss: 1.7379 - val_accuracy: 0.2800 - val_loss: 1.5986
Epoch 2/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 16s 389ms/step - accuracy: 0.2517 - loss: 1.5976 - val_accuracy: 0.3467 - val_loss: 1.5486
Epoch 3/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 10s 435ms/step - accuracy: 0.2646 - loss: 1.5563 - val_accuracy: 0.3467 - val_loss: 1.4948
Epoch 4/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 9s 322ms/step - accuracy: 0.3797 - loss: 1.4530 - val_accuracy: 0.4133 - val_loss: 1.5271
Epoch 5/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 9s 353ms/step - accuracy: 0.3371 - loss: 1.4721 - val_accuracy: 0.3733 - val_loss: 1.4999
Epoch 6/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 8s 279ms/step - accuracy: 0.4311 - loss: 1.2698 - val_accuracy: 0.5333 - val_loss: 1.3493
Epoch 7/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 10s 379ms/step - accuracy: 0.4889 - loss: 1.2029 - val_accuracy: 0.5333 - val_loss: 1.2663
Epoch 8/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 11s 319ms/step - accuracy: 0.4226 - loss: 1.2892 - val_accurac

In [ ]:
# Evaluate the model on the test data
val_loss, val_accuracy = model2.evaluate(val_data, verbose=1)
print(f"val Accuracy: {val_accuracy * 100:.2f}%")
train_loss, train_accuracy = model2.evaluate(train_data, verbose=1)
print(f"train Accuracy: {train_accuracy * 100:.2f}%")
# Evaluate the model on the test data
test_loss, test_accuracy = model2.evaluate(test_data, verbose=1)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 410ms/step - accuracy: 0.7155 - loss: 0.8898
val Accuracy: 72.00%
11/11 ━━━━━━━━━━━━━━━━━━━━ 7s 702ms/step - accuracy: 0.8018 - loss: 0.5290
train Accuracy: 79.14%
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 471ms/step - accuracy: 0.6768 - loss: 0.9359
Test Accuracy: 69.74%


In [ ]:
# Save the model
model2.save('saved_model2.h5')  # Save as HDF5 in Colab's current directory
files.download('saved_model2.h5')  # Downloads the file directly



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>